In [ ]:
from tqdm.auto import tqdm
import os





In [ ]:
with open('/content/laws_clean.txt') as file:
    data = [line.rstrip() for line in file]

In [ ]:
!mkdir model

In [ ]:
text_data = []
file_count = 0

for sample in tqdm(data):
    sample = sample.replace('\n', '')
    text_data.append(sample)
    if len(text_data) == 10_000:
        with open(f'/content/dic/text_{file_count}.txt', 'w', encoding='utf-8') as fp:
            fp.write('\n'.join(text_data))
        text_data = []
        file_count += 1
with open(f'/content/dic/text_{file_count}.txt', 'w', encoding='utf-8') as fp:
    fp.write('\n'.join(text_data))

  0%|          | 0/52853 [00:00<?, ?it/s]

In [ ]:
from pathlib import Path
paths = [str(x) for x in Path('/content/dic/').glob('**/*.txt')]

In [ ]:
paths

['/content/dic/text_0.txt',
 '/content/dic/text_4.txt',
 '/content/dic/text_3.txt',
 '/content/dic/text_5.txt',
 '/content/dic/text_1.txt',
 '/content/dic/text_2.txt']

In [ ]:
!pip install tokenizers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 86.4 MB/s eta 0:00:00


In [ ]:
from tokenizers import ByteLevelBPETokenizer

tokenizer = ByteLevelBPETokenizer()

In [ ]:
tokenizer.train(files=paths[:5], vocab_size=30_522, min_frequency=2,
                special_tokens=['<s>', '<pad>', '</s>', '<unk>', '<mask>'])

In [ ]:
# os.mkdir('content/token-model')

tokenizer.save_model('/content/model')

['/content/model/vocab.json', '/content/model/merges.txt']

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 84.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 25.0 MB/s eta 0:00:00


In [41]:
from transformers import RobertaTokenizerFast
# Create the tokenizer from a trained one
tokenizer = RobertaTokenizerFast.from_pretrained('/content/model', max_len=200)

In [42]:
import pandas as pd

In [45]:
df= pd.read_csv('/content/laws_clean.csv')

In [47]:
train=  df.sample(frac = 0.7)
test = df.drop(train.index)

In [52]:
class CustomDataset():
    def __init__(self, df, tokenizer):
        # or use the RobertaTokenizer from `transformers` directly.
        self.examples = []
        # For every value in the dataframe 
        for example in df.values:
            # 
            x=tokenizer.encode_plus(example, max_length = 200, truncation=True, padding=True)
            self.examples += [x.input_ids]

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, i):
        # We’ll pad at the batch level.
        return torch.tensor(self.examples[i])
      
# Create the train and evaluation dataset
train_dataset = CustomDataset(train['text'], tokenizer)
eval_dataset = CustomDataset(test['text'], tokenizer)

In [53]:
from transformers import DataCollatorForLanguageModeling

# Define the Data Collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [55]:
from transformers import Trainer, TrainingArguments
# Define the training arguments
training_args = TrainingArguments(
    output_dir='/content/model',
    overwrite_output_dir=True,
    evaluation_strategy = 'epoch',
    num_train_epochs=50,
    learning_rate=0.04,
    # weight_decay=10,
    per_device_train_batch_size=100,
    per_device_eval_batch_size=100,
    save_steps=8192,
    #eval_steps=4096,
    save_total_limit=1,
)
# Create the trainer for our model
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    #prediction_loss_only=True,
)
# Train the model
trainer.train()